In [ ]:
import os
import pandas as pd
import gzip
import json

**Reading Meta Data File**

In [ ]:
os.chdir(r'/N/u/abbajpai/Carbonate/Documents/Social Media Mining/data/Raw data') # Change path
metaRawData = []
with gzip.open('meta_Cell_Phones_and_Accessories.json.gz') as f:
    for l in f:
        metaRawData.append(json.loads(l.strip()))

In [ ]:
# convert list into pandas dataframe and check for duplicates
metaData = pd.DataFrame.from_dict(metaRawData) # record count 590071
metaData = metaData.drop_duplicates(subset='asin', keep='first') # 589356
metaData = metaData.dropna(subset=['category']) # no nulls
metaData = metaData[metaData!='[ ]'] #

In [ ]:
# extracting last value in category field 
metaData['categoryLast'] = metaData['category'].astype(str).str.rsplit(',').str[-1]
metaData['categoryLast'] = metaData['categoryLast'].replace('\]', '', regex=True) 
metaData['categoryLast'] = metaData['categoryLast'].replace('\'', '', regex=True)

In [ ]:
# Filering only those asin where final categrory is 'cellphone'
 
metaDataFilter = metaData.query('categoryLast == " Cell Phones" or categoryLast == " Unlocked Cell Phones" or categoryLast == " Carrier Cell Phones"')
# Check                         
print(metaDataFilter['categoryLast'].unique())    
metaDataFilter.to_pickle('1_metaData.pkl') # for future use


**Reading Review Data File**

In [ ]:
'''Step2 : Load and clean Review Data file '''

reviewRawData = []
with gzip.open('Cell_Phones_and_Accessories.json.gz') as f:
    for l in f:
        reviewRawData.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(reviewRawData))

# convert list into pandas dataframe and drop nulls
reviewData = pd.DataFrame.from_dict(reviewRawData)
reviewData= reviewData.dropna(subset=['asin'])
mergedData= pd.merge(reviewData,metaDataFilter, how='inner', on='asin')
mergedData.to_pickle('2_mergedData.pkl')

In [ ]:
print (len (metaRawData))
print (len(metaData))
print (len (metaDataFilter))
print (len(mergedData))